```{=latex}
\usepackage{hyperref}
\usepackage{graphicx}
\usepackage{listings}
\usepackage{textcomp}
\usepackage{fancyvrb}

\newcommand{\passthrough}[1]{\lstset{mathescape=false}#1\lstset{mathescape=true}}
```

```{=latex}
\title{Building Containers for Python Applications}
\author{Moshe Zadka -- https://cobordism.com}
\date{2021}

\begin{document}
\begin{titlepage}
\maketitle
\end{titlepage}

\frame{\titlepage}
```

```{=latex}
\begin{frame}
\frametitle{Acknowledgement of Country}

Belmont (in San Francisco Bay Area Peninsula)

Ancestral homeland of the Ramaytush Ohlone

\end{frame}
```

## Good and Bad

```{=latex}
\begin{frame}
\frametitle{What is good}

\end{frame}
```

```{=latex}
\begin{frame}
\frametitle{Specifying the requirements}

\end{frame}
```

```{=latex}
\begin{frame}
\frametitle{Reliable builds}

\end{frame}
```

```{=latex}
\begin{frame}
\frametitle{Up to date}

\end{frame}
```

```{=latex}
\begin{frame}
\frametitle{No compilers in prod}

\end{frame}
```

```{=latex}
\begin{frame}
\frametitle{Size}

\end{frame}
```

```{=latex}
\begin{frame}
\frametitle{Support binary wheels}

\end{frame}
```

```{=latex}
\begin{frame}
\frametitle{Not run as root}

\end{frame}
```

```{=latex}
\begin{frame}
\frametitle{Minimal privileges}

\end{frame}
```

```{=latex}
\begin{frame}
\frametitle{Fast rebuilds}

\end{frame}
```

## Bases

```{=latex}
\begin{frame}
\frametitle{Considerations - size}

\end{frame}
```

```{=latex}
\begin{frame}
\frametitle{Considerations - LTS/support}

\end{frame}
```

```{=latex}
\begin{frame}
\frametitle{Considerations - volatility}

\end{frame}
```

```{=latex}
\begin{frame}
\frametitle{Debian}

\end{frame}
```

```{=latex}
\begin{frame}
\frametitle{Ubuntu}

\end{frame}
```

```{=latex}
\begin{frame}
\frametitle{Alpine (probably not)}

\end{frame}
```

```{=latex}
\begin{frame}
\frametitle{CentOS (probably not)}

\end{frame}
```

```{=latex}
\begin{frame}
\frametitle{Rolling releases (probably not)}

\end{frame}
```

## Installing Python

* Not system Python
* Appropriate repositories
* pyenv
* python-build
* Source
* Trade-offs
* Versions

## Thinking in Stages

* What is a multistage build?
* FROM
* COPY --from
* Stages a as modules
* Separate build and runtime
* Optimizing layers
* Optimizing size

## Use in Applications

* Binary wheels
* Self-contained binary wheels
* Portable binary wheels
* Patchelf
* Auditwheel
* Generating binary wheels
* Binary wheels and Docker
* Optimizing layers
* Optimizing caching

## Keeping Up

* Security updates
* Tagging strategy
* TTL strategy
* Update process

## Final Thoughts

* Python, Microservices, and Containers
* Trade-offs
* Document choices
* Audit

```{=latex}
\end{document}
```